In [ ]:
# --------------------------------------------------------------#
#                 QUANTIZACIÓN MODELO ONNX                      #
#---------------------------------------------------------------#

import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

model_fp32 = './onnx/model.onnx'
model_quant = './onnx/model.quant.onnx'
quantized_model = quantize_dynamic(model_fp32, model_quant)

In [ ]:
# --------------------------------------------------------------#
#              TESTEO DEL MODELO QUANTIZADO ONNX                #
#---------------------------------------------------------------#

import onnxruntime as ort
import numpy as np
from transformers import AutoTokenizer
import torch
import time
from pprint import pp

# cargamos el modelo y el tokenizer del model cuantizado
session = ort.InferenceSession("./onnx/model.quant.onnx", providers=["CPUExecutionProvider"])
tokenizer = AutoTokenizer.from_pretrained("./onnx")

# Extraemos los comandos para el testeo
text = "avísame en una hora y tres cuartos"

# Los tokenizamos
tokens = tokenizer(text, padding=True, truncation=True, return_tensors="np")  # Convert to NumPy array
#tokens = tokenizer(text, return_tensors="np")  # Convert to NumPy array


# Ensure input tensor shape matches what the model expects
input_ids = tokens["input_ids"].astype(np.int64)  # ONNX models often require int64 inputs
attention_mask = tokens["attention_mask"].astype(np.int64)

# Preparamos los inputs para pasárselos al modelo
inputs = {
    "input_ids": input_ids,
    "attention_mask": attention_mask
}

# Ejecutamos las predicciones y obtenemos los resultados
outputs = session.run(None, inputs)

#print(f"\nOUTPUTS: \n{outputs}")

logits = torch.tensor(outputs[0])  # Convert output to a PyTorch tensor
predictions = torch.argmax(logits, dim=-1)  # Get predicted class index
'''predicted_classes = predictions.tolist()
print(f"Predicted classes: {predicted_classes}")'''

# Print tokens and predictions
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

pp(torch.argmax(logits, dim=-1))

for token, label_id in zip(tokens, predictions[0].tolist()):
    print(f"{token}: {id2tag[label_id]}")